# Set up Model

In [ ]:
# %% Set up the model
# Parameters from the paper: P=0 (number of components), optimal smoothness lambda_sigma ≈ 1.5

# Get the number of neurons from data
N = data.shape[0]  # Number of neurons

# Initialize processes
gp = models.GaussianProcess(kernel=kernel_gp, N=N)
wp = models.WishartProcess(kernel=kernel_wp, P=N+1, V=1e-2*jnp.eye(N))  # P=N+1 as per the code

# Likelihood model
likelihood = models.NormalConditionalLikelihood(N)

# Joint distribution
joint = models.JointGaussianWishartProcess(gp, wp, likelihood)

# %% Cross-validation to select hyperparameters
# This is a simplified version - in practice, you'd want to implement a full
# cross-validation procedure as mentioned in the paper

def perform_cross_validation(x, y, P_values=[0, 1, 2], lambda_values=[0.5, 1.0, 1.5, 2.0]):
    """
    Perform cross-validation to select optimal P and lambda_sigma values.
    
    In a real implementation, you would:
    1. Split data into training and test sets multiple times
    2. For each hyperparameter combination, train on training set
    3. Evaluate log probability on test set
    4. Average results across folds
    """
    print("In practice, you would implement a full cross-validation here.")
    print("Based on the paper, optimal values were P=0 and lambda_sigma ≈ 1.5")
    
    # This would be your actual cross-validation logic
    # For demonstration, we'll just return the values mentioned in the paper
    return 0, 1.5  # P=0, lambda_sigma=1.5

# For demonstration, we'll just use the paper's values
P_optimal, lambda_optimal = perform_cross_validation(x, y)
print(f"Selected hyperparameters: P={P_optimal}, lambda_sigma={lambda_optimal}")

# %% Run inference
# Initialize variational family
varfam = inference.VariationalNormal(joint.model)

# Set up optimizer
adam = optim.Adam(1e-2)  # Learning rate

# Set random seed
inference_seed = 2
key = jax.random.PRNGKey(inference_seed)

# Run variational inference
print("Running variational inference (this may take a while)...")
varfam.infer(adam, x, y, n_iter=5000, key=key)  # Reduced from 20000 for demonstration
joint.update_params(varfam.posterior)

# %% Visualize training loss
visualizations.plot_loss(
    [varfam.losses],
    xlabel='Iteration',
    ylabel='ELBO',
    titlestr='Training Loss',
    colors=['k']
)

# %% Sample from the posterior
posterior = models.NormalGaussianWishartPosterior(joint, varfam, x)

with numpyro.handlers.seed(rng_seed=inference_seed):
    mu_hat, sigma_hat, F_hat = posterior.sample(x)


# Visualise Results

In [ ]:

# %% Visualize results for a few selected neurons/conditions
def plot_correlation_matrix(sigma, condition_idx, title):
    """Plot the correlation matrix for a specific condition"""
    cov_matrix = sigma[condition_idx]
    diag = jnp.sqrt(jnp.diag(cov_matrix))
    corr_matrix = cov_matrix / jnp.outer(diag, diag)
    
    plt.figure(figsize=(8, 6))
    plt.imshow(corr_matrix, cmap='coolwarm', vmin=-1, vmax=1)
    plt.colorbar(label='Correlation')
    plt.title(title)
    plt.xlabel('Neuron')
    plt.ylabel('Neuron')
    
    # Add correlation values as text
    for i in range(corr_matrix.shape[0]):
        for j in range(corr_matrix.shape[1]):
            if i != j:  # Only show off-diagonal elements
                plt.text(j, i, f'{corr_matrix[i, j]:.2f}', 
                         ha='center', va='center', 
                         color='white' if abs(corr_matrix[i, j]) > 0.5 else 'black')
    
    plt.tight_layout()
    plt.show()

# Plot correlation matrices for select conditions
num_conditions = sigma_hat.shape[0]
plot_indices = [0, num_conditions // 2, num_conditions - 1]  # First, middle, last

for idx in plot_indices:
    angle = conditions[idx][0] * 180 / jnp.pi  # Convert to degrees
    sf = conditions[idx][1]
    title = f"Noise Correlation at Angle={angle:.1f}°, SF={sf:.2f}"
    plot_correlation_matrix(sigma_hat, idx, title)


# Compare other things

In [ ]:
# %% Compare with empirical correlations for validation
def compute_empirical_correlation(data, condition_idx):
    """Compute empirical correlation matrix for a specific condition"""
    # Extract data for the condition across all trials
    condition_data = data_model[:, condition_idx, :]  # trials x neurons
    
    # Compute correlation matrix
    corr_matrix = np.corrcoef(condition_data, rowvar=False)
    
    return corr_matrix

# Compare model and empirical correlations for a selected condition
selected_condition = 0
model_corr = sigma_hat[selected_condition]
diag = jnp.sqrt(jnp.diag(model_corr))
model_corr = model_corr / jnp.outer(diag, diag)

empirical_corr = compute_empirical_correlation(data, selected_condition)

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Plot model correlation
im0 = axes[0].imshow(model_corr, cmap='coolwarm', vmin=-1, vmax=1)
axes[0].set_title('Model-Based Correlation')
axes[0].set_xlabel('Neuron')
axes[0].set_ylabel('Neuron')

# Plot empirical correlation
im1 = axes[1].imshow(empirical_corr, cmap='coolwarm', vmin=-1, vmax=1)
axes[1].set_title('Empirical Correlation')
axes[1].set_xlabel('Neuron')

# Add colorbar
fig.colorbar(im0, ax=axes, label='Correlation')

plt.tight_layout()
plt.show()


In [ ]:
# %% Create a function to analyze correlations across conditions
def analyze_correlation_across_conditions(sigma_hat, conditions):
    """Analyze how correlations change across conditions"""
    # Calculate average absolute correlation for each condition
    avg_correlations = []
    
    for i in range(sigma_hat.shape[0]):
        cov_matrix = sigma_hat[i]
        diag = jnp.sqrt(jnp.diag(cov_matrix))
        corr_matrix = cov_matrix / jnp.outer(diag, diag)
        
        # Get off-diagonal elements
        mask = ~jnp.eye(corr_matrix.shape[0], dtype=bool)
        off_diag = corr_matrix[mask]
        
        # Calculate average absolute correlation
        avg_abs_corr = jnp.mean(jnp.abs(off_diag))
        avg_correlations.append(avg_abs_corr)
    
    # Extract angles and spatial frequencies
    angles = conditions[:, 0] * 180 / jnp.pi  # Convert to degrees
    sfs = conditions[:, 1]
    
    # Create a unique identifier for each condition
    condition_ids = [f"{a:.0f}°, SF={s:.2f}" for a, s in zip(angles, sfs)]
    
    # Plot
    plt.figure(figsize=(12, 6))
    plt.bar(range(len(avg_correlations)), avg_correlations)
    plt.xticks(range(len(avg_correlations)), condition_ids, rotation=90)
    plt.ylabel('Average Absolute Correlation')
    plt.title('Noise Correlation Strength Across Conditions')
    plt.tight_layout()
    plt.show()
    
    return avg_correlations

# Run the analysis
avg_correlations = analyze_correlation_across_conditions(sigma_hat, conditions)

In [ ]:
# %% Create a function to extract specific correlation patterns
def extract_correlation_pattern(sigma_hat, conditions, neuron_pair=(0, 1)):
    """Extract correlation between a specific pair of neurons across conditions"""
    n1, n2 = neuron_pair
    
    # Extract correlations
    correlations = []
    for i in range(sigma_hat.shape[0]):
        cov_matrix = sigma_hat[i]
        diag = jnp.sqrt(jnp.diag(cov_matrix))
        corr_matrix = cov_matrix / jnp.outer(diag, diag)
        correlations.append(corr_matrix[n1, n2])
    
    # Reshape to match angles and spatial frequencies grid
    angles_count = data.shape[1]
    sf_count = data.shape[2]
    corr_grid = np.array(correlations).reshape(sf_count, angles_count)
    
    # Create coordinate grids for plotting
    angle_vals = np.linspace(0, 360, angles_count, endpoint=False)
    sf_vals = np.linspace(0.1, 5.0, sf_count)
    angle_grid, sf_grid = np.meshgrid(angle_vals, sf_vals)
    
    # Plot as a heatmap
    plt.figure(figsize=(10, 6))
    plt.pcolormesh(angle_grid, sf_grid, corr_grid, cmap='coolwarm', vmin=-1, vmax=1)
    plt.colorbar(label='Correlation')
    plt.xlabel('Orientation (degrees)')
    plt.ylabel('Spatial Frequency')
    plt.title(f'Noise Correlation between Neurons {n1} and {n2}')
    plt.tight_layout()
    plt.show()
    
    return corr_grid

# Analyze correlation pattern for a pair of neurons
corr_pattern = extract_correlation_pattern(sigma_hat, conditions, neuron_pair=(0, 1))

# %% Save results
def save_results(sigma_hat, mu_hat, conditions):
    """Save results to disk"""
    # Create output directory if it doesn't exist
    if not os.path.exists('results'):
        os.makedirs('results')
    
    # Convert to numpy for saving
    sigma_np = np.array(sigma_hat)
    mu_np = np.array(mu_hat)
    conditions_np = np.array(conditions)
    
    # Save
    np.save('results/sigma_hat.npy', sigma_np)
    np.save('results/mu_hat.npy', mu_np)
    np.save('results/conditions.npy', conditions_np)
    
    print("Results saved to 'results' directory")

# Uncomment to save results:
# save_results(sigma_hat, mu_hat, conditions)

# %% Summary
print("=== Analysis Complete ===")
print(f"- Analyzed data with {data.shape[0]} neurons across {data.shape[1]*data.shape[2]} conditions")
print(f"- Estimated noise correlations using Wishart Process model")
print(f"- Average absolute correlation across all conditions: {np.mean(avg_correlations):.3f}")
print("- Results can be further analyzed by neuron pairs, conditions, or other factors")

# Fisher Information

In [ ]:
# %% Fisher Information Calculation with Wishart Process
# =====================================================
#
# This notebook implements Fisher Information calculation for neural data
# using the posterior distributions from a Wishart Process model.

import jax
import jax.numpy as jnp
from jax import grad, vmap, jacfwd, jacrev
# %% Define Helper Functions for Fisher Information Calculation

def compute_gradient(func, x):
    """
    Compute the gradient of a function with respect to inputs x
    
    Args:
        func: The function to differentiate
        x: The point at which to evaluate the gradient
        
    Returns:
        The gradient of func at x
    """
    return grad(func)(x)

def compute_hessian(func, x):
    """
    Compute the Hessian matrix (second derivatives) of a function
    
    Args:
        func: The function to differentiate
        x: The point at which to evaluate the Hessian
        
    Returns:
        The Hessian matrix of func at x
    """
    return jacfwd(jacrev(func))(x)

# %% Load Posterior Model from Previous Analysis
# This assumes you've run the previous notebook and have saved results

def load_posterior_model(result_path="results"):
    """Load posterior model parameters from saved results"""
    try:
        mu_hat = jnp.array(np.load(os.path.join(result_path, "mu_hat.npy")))
        sigma_hat = jnp.array(np.load(os.path.join(result_path, "sigma_hat.npy")))
        conditions = jnp.array(np.load(os.path.join(result_path, "conditions.npy")))
        print("Loaded saved posterior model")
        return mu_hat, sigma_hat, conditions
    except:
        print("Saved results not found. Generating synthetic data instead.")
        # Create synthetic data for demonstration
        N = 10  # Number of neurons
        C = 20  # Number of conditions
        
        # Create a condition space (e.g., angles from 0 to 2π)
        conditions = jnp.linspace(0, 2*jnp.pi, C).reshape(-1, 1)
        
        # Create synthetic mean responses
        mu_hat = jnp.zeros((C, N))
        for i in range(N):
            mu_hat = mu_hat.at[:, i].set(2*jnp.sin(conditions[:, 0] + i*0.5))
        
        # Create synthetic covariance matrices
        base_sigma = jnp.eye(N) * 0.5
        for i in range(N):
            for j in range(i+1, N):
                base_sigma = base_sigma.at[i, j].set(0.1)
                base_sigma = base_sigma.at[j, i].set(0.1)
        
        sigma_hat = jnp.array([base_sigma * (1 + 0.5*jnp.sin(conditions[i, 0])) for i in range(C)])
        
        return mu_hat, sigma_hat, conditions

In [ ]:

# %% Augment the data with additional interpolated points for better gradient estimation

def interpolate_posterior(mu_hat, sigma_hat, conditions, interp_factor=5):
    """
    Interpolate the posterior distribution to get smoother gradients
    
    Args:
        mu_hat: Mean estimates at original conditions
        sigma_hat: Covariance estimates at original conditions
        conditions: Original condition points
        interp_factor: Factor by which to increase density of points
        
    Returns:
        Interpolated means, covariances, and conditions
    """
    if conditions.shape[1] == 1:  # 1D condition space (e.g., just angles)
        # Create a finer grid of conditions
        min_cond = jnp.min(conditions)
        max_cond = jnp.max(conditions)
        fine_conditions = jnp.linspace(min_cond, max_cond, 
                                       (conditions.shape[0]-1)*interp_factor + 1).reshape(-1, 1)
        
        # Use JAX interpolation for the means (neurons × conditions)
        N = mu_hat.shape[1]  # Number of neurons
        interp_mu = jnp.zeros((fine_conditions.shape[0], N))
        
        # Interpolate each neuron's response
        for n in range(N):
            # Linear interpolation for each neuron
            interp_mu = interp_mu.at[:, n].set(
                jnp.interp(fine_conditions[:, 0], conditions[:, 0], mu_hat[:, n])
            )
        
        # Interpolate covariances
        # This is trickier - for each i,j pair in the covariance matrix
        interp_sigma = jnp.zeros((fine_conditions.shape[0], N, N))
        for i in range(N):
            for j in range(N):
                interp_sigma = interp_sigma.at[:, i, j].set(
                    jnp.interp(fine_conditions[:, 0], conditions[:, 0], sigma_hat[:, i, j])
                )
                
        return interp_mu, interp_sigma, fine_conditions
    
    elif conditions.shape[1] == 2:  # 2D condition space (e.g., angle and spatial frequency)
        print("2D interpolation not implemented in this demo. Using original data.")
        return mu_hat, sigma_hat, conditions
    
    else:
        print("Higher-dimensional interpolation not implemented. Using original data.")
        return mu_hat, sigma_hat, conditions

# %% Compute Fisher Information

def compute_fisher_information_single_point(mu_hat, sigma_hat, mu_grad, sigma_grad):
    """
    Compute Fisher Information at a single condition point
    
    Args:
        mu_hat: Mean vector at this condition
        sigma_hat: Covariance matrix at this condition
        mu_grad: Gradient of mean vector w.r.t. condition
        sigma_grad: Gradient of covariance matrix w.r.t. condition
        
    Returns:
        Fisher Information value at this point
    """
    # Invert covariance matrix
    sigma_inv = jnp.linalg.inv(sigma_hat)
    
    # First term: μ′(x)ᵀ Σ⁻¹(x) μ′(x)
    first_term = mu_grad.T @ sigma_inv @ mu_grad
    
    # Second term: (1/2) tr([Σ⁻¹(x)Σ′(x)]²)
    sigma_inv_sigma_grad = sigma_inv @ sigma_grad
    second_term = 0.5 * jnp.trace(sigma_inv_sigma_grad @ sigma_inv_sigma_grad)
    
    # Total Fisher Information
    fi = first_term + second_term
    
    return fi

def estimate_gradients(mu_hat, sigma_hat, conditions):
    """
    Estimate the gradients of the mean and covariance with respect to conditions
    using finite differences
    
    Args:
        mu_hat: Posterior mean estimates (conditions × neurons)
        sigma_hat: Posterior covariance estimates (conditions × neurons × neurons)
        conditions: Condition points
        
    Returns:
        Gradients of mean and covariance at each condition point
    """
    C, N = mu_hat.shape  # C conditions, N neurons
    
    if conditions.shape[1] == 1:  # 1D condition space
        # Compute gradient using central differences
        mu_grad = jnp.zeros((C, N))
        sigma_grad = jnp.zeros((C, N, N))
        
        # For interior points
        for i in range(1, C-1):
            # Gradient of mean
            mu_grad = mu_grad.at[i].set(
                (mu_hat[i+1] - mu_hat[i-1]) / (conditions[i+1, 0] - conditions[i-1, 0])
            )
            
            # Gradient of covariance
            sigma_grad = sigma_grad.at[i].set(
                (sigma_hat[i+1] - sigma_hat[i-1]) / (conditions[i+1, 0] - conditions[i-1, 0])
            )
        
        # For boundary points, use forward/backward differences
        # First point
        mu_grad = mu_grad.at[0].set(
            (mu_hat[1] - mu_hat[0]) / (conditions[1, 0] - conditions[0, 0])
        )
        sigma_grad = sigma_grad.at[0].set(
            (sigma_hat[1] - sigma_hat[0]) / (conditions[1, 0] - conditions[0, 0])
        )
        
        # Last point
        mu_grad = mu_grad.at[C-1].set(
            (mu_hat[C-1] - mu_hat[C-2]) / (conditions[C-1, 0] - conditions[C-2, 0])
        )
        sigma_grad = sigma_grad.at[C-1].set(
            (sigma_hat[C-1] - sigma_hat[C-2]) / (conditions[C-1, 0] - conditions[C-2, 0])
        )
        
        return mu_grad, sigma_grad
    
    elif conditions.shape[1] == 2:  # 2D condition space
        # For 2D, we need separate gradients for each dimension
        # This would be a more complex implementation
        # For simplicity in this demo, we'll use a placeholder implementation
        print("2D gradient estimation simplification used: computing only along first dimension")
        
        # Simplified: just compute gradient along first dimension 
        # In a real implementation, you would compute the full gradient vector
        mu_grad = jnp.zeros((C, N))
        sigma_grad = jnp.zeros((C, N, N))
        
        # Sort conditions by first dimension
        sort_idx = jnp.argsort(conditions[:, 0])
        sorted_conditions = conditions[sort_idx]
        sorted_mu = mu_hat[sort_idx]
        sorted_sigma = sigma_hat[sort_idx]
        
        # Now compute gradients as in 1D case
        for i in range(1, C-1):
            # Gradient of mean
            mu_grad = mu_grad.at[sort_idx[i]].set(
                (sorted_mu[i+1] - sorted_mu[i-1]) / 
                (sorted_conditions[i+1, 0] - sorted_conditions[i-1, 0])
            )
            
            # Gradient of covariance
            sigma_grad = sigma_grad.at[sort_idx[i]].set(
                (sorted_sigma[i+1] - sorted_sigma[i-1]) / 
                (sorted_conditions[i+1, 0] - sorted_conditions[i-1, 0])
            )
        
        return mu_grad, sigma_grad
    
    else:
        raise ValueError("Higher-dimensional gradient estimation not implemented")

def compute_fisher_information(mu_hat, sigma_hat, conditions):
    """
    Compute Fisher Information across all condition points
    
    Args:
        mu_hat: Posterior mean estimates (conditions × neurons)
        sigma_hat: Posterior covariance estimates (conditions × neurons × neurons)
        conditions: Condition points
        
    Returns:
        Fisher Information at each condition point
    """
    # Get gradients
    mu_grad, sigma_grad = estimate_gradients(mu_hat, sigma_hat, conditions)
    
    # Compute FI at each point
    C = conditions.shape[0]  # Number of conditions
    fi = jnp.zeros(C)
    
    for i in range(C):
        fi = fi.at[i].set(
            compute_fisher_information_single_point(
                mu_hat[i], sigma_hat[i], mu_grad[i], sigma_grad[i]
            )
        )
    
    return fi, mu_grad, sigma_grad

# %% Sample-based Fisher Information with uncertainty

def sample_posterior_gradients(joint, varfam, conditions, n_samples=50):
    """
    Sample from the posterior distribution of mean and covariance gradients
    to estimate uncertainty in Fisher Information
    
    Args:
        joint: Joint Gaussian-Wishart Process model
        varfam: Fitted variational family
        conditions: Condition points
        n_samples: Number of posterior samples
        
    Returns:
        Samples of Fisher Information at each condition point
    """
    print("Not implemented in this demo - requires access to the model posterior")
    
    # In a real implementation, this would:
    # 1. Sample multiple times from the posterior over μ and Σ
    # 2. For each sample, calculate gradients and FI
    # 3. Use the distribution of samples to estimate uncertainty

    # Placeholder
    C = conditions.shape[0]
    return jnp.zeros((n_samples, C))

# %% Main Analysis

# Load or generate data
mu_hat, sigma_hat, conditions = load_posterior_model()

# Interpolate for better gradient estimation
interp_mu, interp_sigma, interp_conditions = interpolate_posterior(mu_hat, sigma_hat, conditions)

# Compute Fisher Information
fi, mu_grad, sigma_grad = compute_fisher_information(interp_mu, interp_sigma, interp_conditions)


In [ ]:
# %% Visualize results
def visualize_fisher_information(fi, conditions):
    """Visualize Fisher Information across conditions"""
    
    if conditions.shape[1] == 1:  # 1D condition space
        # Sort by condition value for proper plotting
        sort_idx = jnp.argsort(conditions[:, 0])
        sorted_fi = fi[sort_idx]
        sorted_conditions = conditions[sort_idx, 0]
        
        plt.figure(figsize=(10, 6))
        plt.plot(sorted_conditions, sorted_fi, 'b-', linewidth=2)
        plt.fill_between(sorted_conditions, 
                         sorted_fi - sorted_fi*0.2, 
                         sorted_fi + sorted_fi*0.2, 
                         alpha=0.3, color='blue',
                         label='Approximate uncertainty')
        
        if sorted_conditions[0] >= 0 and sorted_conditions[-1] <= 2*np.pi + 0.1:
            # If conditions appear to be angles
            plt.xlabel('Angle (radians)')
            plt.xticks(np.linspace(0, 2*np.pi, 5), 
                      ['0', 'π/2', 'π', '3π/2', '2π'])
        else:
            plt.xlabel('Condition')
            
        plt.ylabel('Fisher Information')
        plt.title('Fisher Information Across Conditions')
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.legend()
        plt.show()
        
    elif conditions.shape[1] == 2:  # 2D condition space
        # For 2D conditions like (angle, spatial frequency)
        # Extract unique values for each dimension
        angles = np.unique(conditions[:, 0])
        sfs = np.unique(conditions[:, 1])
        
        # Reshape FI into a grid
        fi_grid = np.zeros((len(sfs), len(angles)))
        
        # Map each condition point to its grid position
        for i, condition in enumerate(conditions):
            angle_idx = np.where(angles == condition[0])[0][0]
            sf_idx = np.where(sfs == condition[1])[0][0]
            fi_grid[sf_idx, angle_idx] = fi[i]
        
        # Create a heatmap
        plt.figure(figsize=(10, 8))
        plt.pcolormesh(angles, sfs, fi_grid, cmap='viridis', shading='auto')
        plt.colorbar(label='Fisher Information')
        
        if angles[0] >= 0 and angles[-1] <= 2*np.pi + 0.1:
            # If first dimension appears to be angles
            plt.xlabel('Angle (radians)')
            plt.xticks(np.linspace(0, 2*np.pi, 5), 
                      ['0', 'π/2', 'π', '3π/2', '2π'])
        else:
            plt.xlabel('Condition Dimension 1')
            
        plt.ylabel('Spatial Frequency' if conditions.shape[1] == 2 else 'Condition Dimension 2')
        plt.title('Fisher Information Across Condition Space')
        plt.show()
    
    else:
        print(f"Visualization for {conditions.shape[1]}D condition space not implemented")

# Visualize Fisher Information
visualize_fisher_information(fi, interp_conditions)




In [ ]:
# %% Fisher Information Integration for Discriminability

def analyze_discriminability(fi, conditions, dimension=0):
    """
    Analyze discriminability between condition pairs based on Fisher Information
    
    Args:
        fi: Fisher Information at each condition
        conditions: Condition points
        dimension: Dimension along which to compute discriminability (for multi-dimensional conditions)
        
    Returns:
        Discriminability matrix showing JND (Just Noticeable Difference) between conditions
    """
    # Extract conditions along the specified dimension
    cond_values = conditions[:, dimension]
    
    # Sort by condition value
    sort_idx = jnp.argsort(cond_values)
    sorted_fi = fi[sort_idx]
    sorted_conds = cond_values[sort_idx]
    
    # Compute discriminability matrix based on integrated Fisher Information
    C = len(sorted_conds)
    disc_matrix = jnp.zeros((C, C))
    
    for i in range(C):
        for j in range(i+1, C):
            # For each pair of conditions, compute integrated FI
            # We use a simple trapezoidal integration here
            segment_indices = list(range(i, j+1))
            segment_fi = sorted_fi[segment_indices]
            segment_conds = sorted_conds[segment_indices]
            
            # Integrate FI along the path from condition i to j
            # d'² = ∫ FI(x) dx between i and j
            disc_squared = 0
            for k in range(len(segment_indices)-1):
                # Trapezoidal rule
                avg_fi = (segment_fi[k] + segment_fi[k+1]) / 2
                delta_x = segment_conds[k+1] - segment_conds[k]
                disc_squared += avg_fi * delta_x
            
            # Convert to discriminability (d')
            discriminability = jnp.sqrt(disc_squared)
            disc_matrix = disc_matrix.at[i, j].set(discriminability)
            disc_matrix = disc_matrix.at[j, i].set(discriminability)  # Symmetric
    
    # Convert to JND (Just Noticeable Difference)
    # Define a threshold for discriminability (e.g., d'=1)
    threshold = 1.0
    jnd_matrix = disc_matrix / threshold
    
    # Visualize the discriminability matrix
    plt.figure(figsize=(10, 8))
    plt.imshow(jnd_matrix, origin='lower', cmap='viridis')
    plt.colorbar(label='JND (d\'/threshold)')
    plt.xlabel('Condition Index')
    plt.ylabel('Condition Index')
    plt.title('Just Noticeable Difference Between Conditions')
    
    # Add condition values to ticks (if not too many)
    if C <= 10:
        plt.xticks(range(C), [f"{x:.2f}" for x in sorted_conds])
        plt.yticks(range(C), [f"{x:.2f}" for x in sorted_conds])
    else:
        # Show only a subset of ticks
        tick_indices = np.linspace(0, C-1, 5, dtype=int)
        plt.xticks(tick_indices, [f"{sorted_conds[i]:.2f}" for i in tick_indices])
        plt.yticks(tick_indices, [f"{sorted_conds[i]:.2f}" for i in tick_indices])
    
    plt.tight_layout()
    plt.show()
    
    return jnd_matrix

# Analyze discriminability
jnd_matrix = analyze_discriminability(fi, interp_conditions)



In [ ]:

# %% Analyze contribution of mean gradients vs. covariance gradients

def analyze_fi_components(mu_hat, sigma_hat, mu_grad, sigma_grad, conditions):
    """
    Analyze the separate contributions to Fisher Information from
    mean gradients and covariance gradients
    """
    C = conditions.shape[0]
    fi_mean = jnp.zeros(C)  # Component from mean gradients
    fi_cov = jnp.zeros(C)   # Component from covariance gradients
    
    for i in range(C):
        # Contribution from mean gradients: μ′(x)ᵀ Σ⁻¹(x) μ′(x)
        sigma_inv = jnp.linalg.inv(sigma_hat[i])
        fi_mean = fi_mean.at[i].set(mu_grad[i].T @ sigma_inv @ mu_grad[i])
        
        # Contribution from covariance gradients: (1/2) tr([Σ⁻¹(x)Σ′(x)]²)
        sigma_inv_sigma_grad = sigma_inv @ sigma_grad[i]
        fi_cov = fi_cov.at[i].set(0.5 * jnp.trace(sigma_inv_sigma_grad @ sigma_inv_sigma_grad))
    
    # Sort by condition for proper plotting
    if conditions.shape[1] == 1:  # 1D condition space
        sort_idx = jnp.argsort(conditions[:, 0])
        sorted_fi_mean = fi_mean[sort_idx]
        sorted_fi_cov = fi_cov[sort_idx]
        sorted_fi_total = sorted_fi_mean + sorted_fi_cov
        sorted_conditions = conditions[sort_idx, 0]
        
        # Plot components
        plt.figure(figsize=(10, 6))
        plt.plot(sorted_conditions, sorted_fi_total, 'k-', linewidth=2, label='Total FI')
        plt.plot(sorted_conditions, sorted_fi_mean, 'b--', linewidth=2, label='Mean component')
        plt.plot(sorted_conditions, sorted_fi_cov, 'r--', linewidth=2, label='Covariance component')
        
        if sorted_conditions[0] >= 0 and sorted_conditions[-1] <= 2*np.pi + 0.1:
            # If conditions appear to be angles
            plt.xlabel('Angle (radians)')
            plt.xticks(np.linspace(0, 2*np.pi, 5), 
                      ['0', 'π/2', 'π', '3π/2', '2π'])
        else:
            plt.xlabel('Condition')
            
        plt.ylabel('Fisher Information')
        plt.title('Components of Fisher Information')
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.legend()
        plt.show()
        
        # Show relative contribution
        plt.figure(figsize=(10, 6))
        plt.stackplot(sorted_conditions, 
                     [sorted_fi_mean / sorted_fi_total, sorted_fi_cov / sorted_fi_total],
                     labels=['Mean component', 'Covariance component'],
                     colors=['blue', 'red'], alpha=0.7)
        
        if sorted_conditions[0] >= 0 and sorted_conditions[-1] <= 2*np.pi + 0.1:
            plt.xlabel('Angle (radians)')
            plt.xticks(np.linspace(0, 2*np.pi, 5), 
                      ['0', 'π/2', 'π', '3π/2', '2π'])
        else:
            plt.xlabel('Condition')
            
        plt.ylabel('Relative Contribution')
        plt.title('Relative Contribution to Fisher Information')
        plt.ylim(0, 1)
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.legend(loc='upper right')
        plt.show()
    
    return fi_mean, fi_cov

# Analyze components
fi_mean, fi_cov = analyze_fi_components(interp_mu, interp_sigma, mu_grad, sigma_grad, interp_conditions)

In [ ]:
# %% Compare Fisher Information under different noise correlation assumptions

def compare_fi_correlation_impact(mu_hat, sigma_hat, mu_grad, sigma_grad, conditions):
    """
    Compare Fisher Information under different noise correlation assumptions:
    - Full covariance (with correlations)
    - Diagonal covariance (no correlations)
    - Uniform correlation (same correlation for all pairs)
    """
    C = conditions.shape[0]
    N = mu_hat.shape[1]  # Number of neurons
    
    fi_full = jnp.zeros(C)      # Original FI with full correlation structure
    fi_diag = jnp.zeros(C)      # FI with diagonal covariance (no correlations)
    fi_uniform = jnp.zeros(C)   # FI with uniform correlation
    
    # Typical uniform correlation value (could be computed from data)
    r_uniform = 0.2
    
    for i in range(C):
        # Full correlation FI (original)
        sigma_inv_full = jnp.linalg.inv(sigma_hat[i])
        fi_full = fi_full.at[i].set(mu_grad[i].T @ sigma_inv_full @ mu_grad[i])
        
        # Diagonal covariance (no correlations)
        sigma_diag = jnp.diag(jnp.diag(sigma_hat[i]))
        sigma_inv_diag = jnp.linalg.inv(sigma_diag)
        fi_diag = fi_diag.at[i].set(mu_grad[i].T @ sigma_inv_diag @ mu_grad[i])
        
        # Uniform correlation
        variances = jnp.diag(sigma_hat[i])
        sigma_uniform = jnp.zeros_like(sigma_hat[i])
        for j in range(N):
            for k in range(N):
                if j == k:
                    sigma_uniform = sigma_uniform.at[j, k].set(variances[j])
                else:
                    sigma_uniform = sigma_uniform.at[j, k].set(r_uniform * jnp.sqrt(variances[j] * variances[k]))
        
        sigma_inv_uniform = jnp.linalg.inv(sigma_uniform)
        fi_uniform = fi_uniform.at[i].set(mu_grad[i].T @ sigma_inv_uniform @ mu_grad[i])
    
    # Sort by condition for proper plotting
    if conditions.shape[1] == 1:  # 1D condition space
        sort_idx = jnp.argsort(conditions[:, 0])
        sorted_fi_full = fi_full[sort_idx]
        sorted_fi_diag = fi_diag[sort_idx]
        sorted_fi_uniform = fi_uniform[sort_idx]
        sorted_conditions = conditions[sort_idx, 0]
        
        # Plot comparison
        plt.figure(figsize=(10, 6))
        plt.plot(sorted_conditions, sorted_fi_full, 'b-', linewidth=2, label='Full correlations')
        plt.plot(sorted_conditions, sorted_fi_diag, 'r--', linewidth=2, label='No correlations')
        plt.plot(sorted_conditions, sorted_fi_uniform, 'g-.', linewidth=2, label=f'Uniform correlation (r={r_uniform})')
        
        if sorted_conditions[0] >= 0 and sorted_conditions[-1] <= 2*np.pi + 0.1:
            # If conditions appear to be angles
            plt.xlabel('Angle (radians)')
            plt.xticks(np.linspace(0, 2*np.pi, 5), 
                      ['0', 'π/2', 'π', '3π/2', '2π'])
        else:
            plt.xlabel('Condition')
            
        plt.ylabel('Fisher Information')
        plt.title('Impact of Noise Correlations on Fisher Information')
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.legend()
        plt.show()
        
        # Plot relative FI
        plt.figure(figsize=(10, 6))
        plt.plot(sorted_conditions, sorted_fi_diag / sorted_fi_full, 'r--', linewidth=2, 
                label='No correlations / Full correlations')
        plt.plot(sorted_conditions, sorted_fi_uniform / sorted_fi_full, 'g-.', linewidth=2, 
                label=f'Uniform correlation / Full correlations')
        plt.axhline(y=1, color='k', linestyle='-', alpha=0.5)
        
        if sorted_conditions[0] >= 0 and sorted_conditions[-1] <= 2*np.pi + 0.1:
            plt.xlabel('Angle (radians)')
            plt.xticks(np.linspace(0, 2*np.pi, 5), 
                      ['0', 'π/2', 'π', '3π/2', '2π'])
        else:
            plt.xlabel('Condition')
            
        plt.ylabel('Relative Fisher Information')
        plt.title('Relative Impact of Noise Correlations')
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.legend()
        plt.show()
    
    return fi_full, fi_diag, fi_uniform

# Compare FI under different correlation assumptions
fi_full, fi_diag, fi_uniform = compare_fi_correlation_impact(
    interp_mu, interp_sigma, mu_grad, sigma_grad, interp_conditions
)

# %% Summary of analysis

print("=== Fisher Information Analysis Complete ===")
print(f"- Analyzed data across {interp_conditions.shape[0]} conditions")
print(f"- Peak Fisher Information: {jnp.max(fi):.4f}")
print(f"- Average Fisher Information: {jnp.mean(fi):.4f}")
print("\nKey Findings:")
print(f"- Mean Contribution: {jnp.mean(fi_mean/fi*100):.1f}% of total FI on average")
print(f"- Covariance Contribution: {jnp.mean(fi_cov/fi*100):.1f}% of total FI on average")
print(f"- Impact of removing correlations: {jnp.mean(fi_diag/fi_full):.2f}x change in FI on average")

# %% Save results
def save_fi_results(fi, conditions, prefix="fi_results"):
    """Save Fisher Information results to disk"""
    # Create output directory if it doesn't exist
    if not os.path.exists('results'):
        os.makedirs('results')
    
    # Convert to numpy for saving
    fi_np = np.array(fi)
    conditions_np = np.array(conditions)
    
    # Save
    np.save(f'results/{prefix}_fi.npy', fi_np)
    np.save(f'results/{